## Inference guide

In [1]:
import os
import shutil
import os.path as osp

In [2]:
EXP_NAME = 'fold_3-mask2former-city_21k-large-epoch_89'
CKPT_PATH = "../runs/mask2former/ku3nqmrl/checkpoints/fold_3-mask2former-city_21k-large-Mask2former-epoch=89-val_high_vegetation__IoU=89.7000.ckpt"
CFG_PATH = ('../configs/mask2former/mask2former-city-21k.yml')


In [3]:
CKPT_NAME = osp.basename(CKPT_PATH)
CFG_NAME = osp.basename(CFG_PATH)

osp.exists(CKPT_PATH), osp.exists(CFG_PATH), CKPT_NAME, CFG_NAME


(True,
 True,
 'fold_3-mask2former-city_21k-large-Mask2former-epoch=89-val_high_vegetation__IoU=89.7000.ckpt',
 'mask2former-city-21k.yml')

In [4]:
os.makedirs(EXP_NAME, exist_ok=True)
shutil.copy(CKPT_PATH, osp.join(EXP_NAME, 'best.ckpt'))
shutil.copy(CFG_PATH, EXP_NAME)

'fold_3-mask2former-city_21k-large-epoch_89/mask2former-city-21k.yml'

In [5]:
# create inference config
TEST_IMG_DIR = "../data/private/img/test/"

config = f"""
global:
  pretrained: null
  save_path: "./result.json"
  batch_size: 1
  
data:
  SIZE: null # ORIGINAL SIZE
  IMG_DIR: {TEST_IMG_DIR}

model:
  name: Mask2former
  args:
    NUM_CLASS: 2
    PRETRAINED: "facebook/mask2former-swin-large-cityscapes-semantic"
    freeze_epochs: -1
    
loss:
  weights:
    bce: 1.0
  smooth_factor: 0.2
"""
with open(f'./{EXP_NAME}/inference.yml', 'w') as f:
    f.write(config)


In [6]:
!ucc-pred   -c  ./{EXP_NAME}/inference.yml \
            -o  global.pretrained=./{EXP_NAME}/best.ckpt\
                data.IMG_DIR={TEST_IMG_DIR} \
                global.save_path=./{EXP_NAME}/results.json

Overriding configurating
Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-base-ade-semantic and are newly initialized because the shapes did not match:
- model.transformer_module.queries_embedder.weight: found shape torch.Size([100, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- model.transformer_module.queries_features.weight: found shape torch.Size([100, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[WARNING] learning rate is not defined, auto set to 1e-3
Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-large-cityscapes-semantic and are newly initialized because the shapes did not match:
- model.transformer_module.queries_embedder.weight: found shape torch.Size(

In [7]:
import json 
import os
with open(f'./{EXP_NAME}/results.json', 'r') as f:
    j = json.loads(f.read())
assert len(j) == len(os.listdir(TEST_IMG_DIR))

In [8]:
%cd {EXP_NAME}
!rm -rf results.zip
!zip results.zip results.json
%cd -

/home/tuantran/ucc-ai-quest-2023/notebooks/fold_3-mask2former-city_21k-large-epoch_89
  adding: results.json (deflated 64%)
/home/tuantran/ucc-ai-quest-2023/notebooks
